In [1]:
import yaml
import numpy as np
import os
import soundfile as sf
from scipy.signal import stft
from tqdm import tqdm
import torch
# If you have a GPU, put the data on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
directory = "/Users/odysseaslazaridis/Documents/GroupProject/babyslakh_16k"
listdir = os.listdir(directory)
listdir.sort()
format = '.wav'
savedir = 'data'
sample_rate = 16000
nperseg = 256
max_size = 250

for i, tr in enumerate(tqdm(listdir)):
    if tr != ".DS_Store":
        tr_dicts_2 = {}
        tr_path = os.path.join(directory, tr)
        with open(os.path.join(tr_path, "metadata.yaml")) as meta:
            metadata = yaml.load(meta, Loader=yaml.Loader)
        # add Mix to the dictionary
        f, t, Zxx = stft(sf.read(os.path.join(directory,tr, 'mix'+format))[0], fs=sample_rate, nperseg=nperseg)
        tr_dicts_2.update({'mix': torch.tensor(Zxx.copy(), dtype=torch.complex128, device=device)})
        file_inst = []
        for stem in metadata['stems'].keys():
            file_inst.append([stem + format, metadata['stems'][stem]['inst_class']])
        file_inst = np.array(file_inst)
        file_inst = file_inst[[file_inst[:,0][i] in os.listdir(tr_path + '/stems') for i in range(len(file_inst))]]
        # combine all stems from the same instrument and track using soundfile
        for inst in np.unique(file_inst[:,1]):
            inst_files = file_inst[file_inst[:,1] == inst][:,0]
            inst_files = [os.path.join(tr_path, 'stems', inst_file) for inst_file in inst_files]
            inst_data = np.array([sf.read(inst_file)[0] for inst_file in inst_files])
            inst_data = np.sum(inst_data, axis=0)
            # compute the STFT of the combined data
            if inst in ['mix','Bass', 'Guitar', 'Drums', 'Piano']:
                f, t, Zxx = stft(inst_data, fs=sample_rate, nperseg=nperseg)
                tr_dicts_2.update({inst: torch.tensor(Zxx.copy(), dtype=torch.complex128, device=device)})
        # save the dictionary as a .pt file
        torch.save(tr_dicts_2, os.path.join(savedir, tr + '.pt'))
        if i > max_size:
            break

100%|██████████| 21/21 [00:20<00:00,  1.00it/s]


In [40]:
tr_dicts_3 = torch.load('/Users/odysseaslazaridis/Documents/GroupProject/new_babyslack/Track000017/Track000017.pt')



FileNotFoundError: [Errno 2] No such file or directory: '/Users/odysseaslazaridis/Documents/GroupProject/new_babyslack/Track00008/Track00006.pt'

In [39]:
tr_dicts_3

{'.DS_Store': {},
 'Track00006': {0: {'Guitar.wav': 1,
   'Piano.wav': 1,
   'Bass.wav': 1,
   'Drums.wav': 1},
  1: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  2: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  3: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  4: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  5: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  6: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  7: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  8: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  9: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  10: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  11: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  12: {'Guitar.wav': 1, 'Piano.wav': 1, 'Bass.wav': 1, 'Drums.wav': 1},
  13: {'Guitar.wav': 1, 'Piano.wa

In [ ]:
for tr in tr_dicts_3:
